In [1]:
import torch
torch.set_printoptions(edgeitems=2, precision=2)

p, q = 6, 16
v, n, m = 3, 2, 4
A = torch.abs(torch.randn(p, q))
A

tensor([[0.74, 0.57, 1.35, 0.84, 2.20, 1.44, 1.31, 0.52, 0.76, 0.85, 0.44, 2.30,
         0.16, 0.68, 1.82, 0.32],
        [0.68, 0.34, 0.75, 0.36, 1.76, 0.27, 1.69, 1.65, 0.49, 0.34, 0.57, 0.10,
         0.05, 0.87, 1.11, 1.07],
        [1.48, 1.73, 0.05, 0.25, 1.12, 0.19, 0.54, 0.91, 0.36, 0.92, 0.27, 0.95,
         0.18, 0.68, 0.19, 0.45],
        [1.05, 0.38, 0.29, 0.27, 0.32, 0.05, 0.26, 0.51, 0.35, 1.46, 1.54, 0.63,
         0.80, 1.76, 0.20, 1.30],
        [0.46, 0.03, 0.37, 0.01, 0.32, 1.78, 1.00, 0.69, 0.18, 0.14, 0.58, 0.19,
         1.07, 1.76, 0.71, 0.26],
        [2.04, 1.44, 1.23, 2.43, 0.36, 0.48, 2.80, 0.43, 1.37, 1.56, 1.33, 0.44,
         1.38, 0.29, 0.35, 1.11]])

In [2]:
def prune_vnm(A, v, n, m, shuffle_cols=False):
    num_blocks = p // v * q // m
    score = A.reshape(p // v, v, q // m, m).permute(0, 2, 1, 3).reshape(num_blocks, v, m)
    
    if shuffle_cols:
        # rand_perm = torch.argsort(torch.randn(num_blocks, 1, m), dim=-1).repeat(1, v, 1)
        rand_perm = torch.argsort(torch.rand(num_blocks, 1, m), dim=-1).repeat(1, v, 1)
        score = torch.gather(score, -1, rand_perm)

    combs = torch.combinations(torch.arange(m), r=2*n)
    combed_score = score.permute(0, 2, 1)[:, combs, :].permute(0, 1, 3, 2).reshape(num_blocks, -1, v, 2*n)

    topk_val, topk_idx = torch.topk(combed_score, k=n, dim=-1)

    sumns = topk_val.sum(-1).sum(-1)

    top_comb_idx = torch.argmax(sumns, dim=-1)

    selected_comb = combs[top_comb_idx]
    top_comb_nm_idx = topk_idx[torch.arange(num_blocks), top_comb_idx]

    merged_idx = torch.gather(
        selected_comb.unsqueeze(1).repeat(1, v, 1),
        -1,
        top_comb_nm_idx
    )

    mask = torch.zeros(num_blocks, v, m, dtype=torch.bool)
    mask.scatter_(-1, merged_idx, 1)

    if shuffle_cols:
        mask = torch.gather(mask, -1, torch.argsort(rand_perm, dim=-1))

    mask = mask.reshape(p // v, q // m, v, m).permute(0, 2, 1, 3).reshape(p, q)
    return mask

In [91]:
AP = A * prune_vnm(A, v, n, 2 * m)

In [3]:
prune_vnm(A, v, n, m, shuffle_cols=False).int()

tensor([[0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0],
        [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1],
        [1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
        [1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1],
        [1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0],
        [1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1]], dtype=torch.int32)

In [4]:
prune_vnm(A, v, n, m, shuffle_cols=True).int()

tensor([[0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0],
        [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1],
        [1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
        [1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1],
        [1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0],
        [1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1]], dtype=torch.int32)

In [52]:
score

tensor([[[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.65, 2.55],
         [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.49, 0.00],
         [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.04, 0.00]],

        [[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
         [0.00, 0.00, 0.00, 0.00, 1.65, 0.00, 0.00, 0.00],
         [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.99]],

        [[0.00, 1.96, 0.00, 1.56, 0.00, 0.00, 0.00, 0.00],
         [0.00, 0.98, 0.00, 0.00, 0.00, 0.00, 0.00, 1.64],
         [0.00, 1.80, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00]],

        [[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
         [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
         [2.01, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00]]])

tensor([[[3, 7, 1, 0, 5, 4, 2, 6],
         [3, 7, 1, 0, 5, 4, 2, 6],
         [3, 7, 1, 0, 5, 4, 2, 6]],

        [[3, 6, 7, 2, 0, 4, 5, 1],
         [3, 6, 7, 2, 0, 4, 5, 1],
         [3, 6, 7, 2, 0, 4, 5, 1]],

        [[6, 3, 0, 7, 2, 5, 1, 4],
         [6, 3, 0, 7, 2, 5, 1, 4],
         [6, 3, 0, 7, 2, 5, 1, 4]],

        [[0, 5, 2, 1, 3, 6, 4, 7],
         [0, 5, 2, 1, 3, 6, 4, 7],
         [0, 5, 2, 1, 3, 6, 4, 7]]])

In [1]:
import torch

In [2]:
state_dict = torch.load('output_dir/bert_base_uncased_acdcpp_vnm_64-2-8_64-2-16_w8kf8kc9k/ckpt/ep0-ba10-rank0.pt')

In [6]:
W = state_dict['state']['model']['model.bert.encoder.layer.2.attention.self.query.weight']

In [10]:
p, q = 768, 768
v, n, m = 64, 2, 16
num_blocks = p // v * q // m
W_blocked = W.reshape(p // v, v, q // m, m).permute(0, 2, 1, 3).reshape(num_blocks, v, m)
W_blocked.shape

torch.Size([576, 64, 16])

In [15]:
(W_blocked[0] != 0).int()

tensor([[0, 0, 0,  ..., 0, 1, 0],
        [0, 0, 0,  ..., 0, 1, 0],
        [0, 0, 0,  ..., 0, 1, 0],
        ...,
        [0, 0, 1,  ..., 0, 0, 0],
        [0, 0, 1,  ..., 0, 0, 0],
        [0, 0, 1,  ..., 0, 1, 0]], device='cuda:0', dtype=torch.int32)